In [ ]:
import numpy as np
import pandas as pd
import pathlib
from tqdm.auto import tqdm

import ray

In [ ]:
import os
os.path.abspath("Data/StudyData/BiHealth_20230313.dta")

In [ ]:
base_path = "/home/jakobs"

In [ ]:
endpoints_md = pd.read_csv(f"{base_path}/BiHealth/onnx/endpoints.csv").drop(columns="Unnamed: 0").set_index("endpoint")#[["endpoint", "eligable", "n", "freq", "phecode", "phecode_string", "phecode_category", "sex"]]
endpoints_md.reset_index().to_feather(f"{base_path}/data/endpoints_md.feather")

In [ ]:
data_images_raw = pd.read_stata(f"{base_path}/BiHealth/Data/StudyData/BiHealth_20230313_Long.dta").sort_values("bihid").reset_index(drop=True).rename(columns={"bihid":"eid"})
data_images_raw

In [ ]:
import os
import pandas as pd

# set the directory paths
folder1_path = f"{base_path}/BiHealth/Data/EPICImages"
folder2_path = f"{base_path}/BiHealth/Data/EPICImages_PoorQuality"

# create an empty list to store the file paths
file_paths = []

# iterate over the folders and append file paths to the list
for folder_path in [folder1_path, folder2_path]:
    for filename in os.listdir(folder_path):
        if filename.endswith(".png"):
            file_paths.append((folder_path, filename))

# create a DataFrame from the list of file paths
df = pd.DataFrame(file_paths, columns=["Folder", "distfilename"])
df["Folder"] = df["Folder"].str.replace("/home/jakobs/BiHealth/Data/EPICImages", "").str.replace("_", "").str.replace("Quality", "")
df["quality"] = df["Folder"]
df = df[["distfilename", "quality"]]

# display the result
df

In [ ]:
data_images = data_images_raw.merge(df, how="left", on="distfilename").query("quality!='Poor'").reset_index(drop=True)
data_images

In [ ]:
data_images.to_feather(f"{base_path}/data/data_images_231117.feather")

In [ ]:
eids_excluded = ['RID02605']

In [ ]:
eids_images = [eid for eid in data_images.eid.unique().tolist() if not eid in eids_excluded]

In [ ]:
data_baseline = pd.read_stata(f"{base_path}/BiHealth/Data/StudyData/BiHealth_20230313.dta").rename(columns={"bihid": "eid", "hc3date":"recruitment_date"}).query("eid==@eids_images").reset_index(drop=True)

In [ ]:
data_baseline

In [ ]:
data_baseline.to_feather(f"{base_path}/data/data_baseline_231117.feather")